In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

In [52]:
# Importing our data
data = pd.read_excel('table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls', 
                     encoding='latin1', skiprows=list(range(4)))

# Condense and clean our dataset
data = data.rename(index = str, columns={'Murder and\nnonnegligent\nmanslaughter': 'Murder', 'Property\ncrime': "Property_crime"})
propertycrime = data[['City', 'Population', 'Murder', 'Robbery', 'Property_crime', 'Burglary']]
propertycrime = propertycrime.dropna()

# Drop outlier, New York
propertycrime = propertycrime.drop(propertycrime.index[216])

# Create the new rows for our regression model
propertycrime['Pop_squared'] = propertycrime['Population']**2

# Preview the data
propertycrime.head()

,City,Population,Murder,Robbery,Property_crime,Burglary,Pop_squared
0,Adams Village,1861.000,0.000,0.000,12.000,2.000,3463321.000
1,Addison Town and Village,2577.000,0.000,0.000,24.000,3.000,6640929.000
2,Akron Village,2846.000,0.000,0.000,16.000,1.000,8099716.000
3,Albany,97956.000,8.000,227.000,4090.000,705.000,9595377936.000
4,Albion Village,6388.000,0.000,4.000,223.000,53.000,40806544.000


In [46]:
# View the column statistics
propertycrime.describe()

,Population,Murder,Robbery,Property_crime,Pop_squared
count,347.000,347.000,347.000,347.000,347.000
mean,15956.686,0.605,17.867,385.752,985840709.758
std,27080.219,3.707,94.972,1034.369,5067232380.434
min,526.000,0.000,0.000,0.000,276676.000
25%,2997.000,0.000,0.000,40.000,8982153.000
50%,7187.000,0.000,1.000,112.000,51652969.000
75%,18160.500,0.000,5.000,340.500,329804222.500
max,258789.000,47.000,1322.000,12491.000,66971746521.000


In [54]:
# Construct our linear regression model with our X variables 
# and predict Y
regr = linear_model.LinearRegression()
X = propertycrime[['Population', 'Murder', 'Robbery', 'Pop_squared']]
Y = propertycrime['Property_crime'].values.reshape(-1, 1)
regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[ 2.05519766e-02  1.02643381e+02  5.13001388e+00 -7.19468539e-08]]

Intercept: 
 [-25.03979961]

R-squared:
0.939283140822424


In [51]:
propertycrime.columns

Index(['City', 'Population', 'Murder', 'Robbery', 'Property_crime',
       'Pop_squared'],
      dtype='object')

In [53]:
X = propertycrime[['Population', 'Murder', 'Robbery', 'Pop_squared','Burglary']]
regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[ 1.20229958e-02  1.58813353e+00 -1.62410781e+00 -1.64297690e-08
   3.62617711e+00]]

Intercept: 
 [-23.55210522]

R-squared:
0.9805504756378403


In [55]:
# Review the correlation matrix
correlation_matrix = X.corr()
display(correlation_matrix)

,Population,Murder,Robbery,Pop_squared
Population,1.000,0.756,0.816,0.889
Murder,0.756,1.000,0.963,0.884
Robbery,0.816,0.963,1.000,0.940
Pop_squared,0.889,0.884,0.940,1.000


# PCA

In [6]:
pca = PCA(n_components=1)
X = StandardScaler().fit_transform(X)
principalComponents = pca.fit_transform(X)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1'])
principalDf.head()

,principal component 1
0,-0.526
1,-0.513
2,-0.508
3,4.441
4,-0.420


In [7]:
# How much of our variance do we retain after limiting our data to 1 component?
pca.explained_variance_ratio_

array([0.90671722])

In [8]:
# Create a new linear regression model with our 1 variable
X = principalDf
Y = propertycrime['Property_crime'].values.reshape(-1, 1)
regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))


Coefficients: 
 [[516.93434211]]

Intercept: 
 [385.75216138]

R-squared:
0.9084581990068199


In [24]:
data = principalDf['principal component 1'].apply(lambda x: x*regr.coef_+regr.intercept_)
target = propertycrime['Property_crime']
data.head()

0     [[114.0539789126928]]
1    [[120.73433640097852]]
2    [[123.25799790403505]]
3    [[2681.3815741998123]]
4    [[168.38930500239923]]
Name: principal component 1, dtype: object

In [25]:
target.head()

0     12.000
1     24.000
2     16.000
3   4090.000
4    223.000
Name: Property_crime, dtype: float64

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.20)
print('With 20% Holdout: ' + str(bnb.fit(X_train, y_train).score(X_test, y_test)))
print('Testing on Sample: ' + str(bnb.fit(data, target).score(data, target)))

ValueError: Expected 2D array, got 1D array instead:
array=[ 369.21753535  916.5592435   249.12852175  113.98893294 2061.57318113
  128.98047322  276.16262008  112.44791651  122.0280571   259.38073181
  248.3848657   131.15886341  699.77580291  111.83602433  160.25597475
  228.04841019  137.72128201  115.8400961   204.98028175  188.52754389
  345.39203664  159.13955896  154.18726401  361.98500623  312.2028729
  177.66209342  547.26318797 1219.53092364  170.03852821  439.63644351
  110.12330137  121.54033417  153.82089084  111.89163206  120.75307175
  137.58794604  284.26709309  175.24639931  451.5187258   156.00992734
  256.23853881  205.72359099  132.85804949  298.9833584   187.18556615
  120.98729887  237.11169104  126.67717257  200.80041263  156.16635707
  123.94544027  221.26897068  484.98849799  162.83675291  377.54551438
  790.92059416  152.74169842  260.58721057  169.93902958  143.89305833
  223.12138654  136.88351088  118.37703171  121.04509477  109.31909673
  154.54007593  226

# Writeup

The original features used to predict property crime were population, population squared, murder, and robbery rates. With the first regression model, our model turns out to have a r-squared value of 0.939, a very good score for our model, and an indication that we have picked the correct variables to predict with. Reducing the complexity of the murder and robbery column as we did with the previous exercise (changing values greater than 0 to 1) will decrease our r-squared value, so we're better off leaving the data as it is. Removing features will also reduce the accuracy of our model, while adding more variables (such as burglary) will increase it. This makes sense since adding more variables into the equation should provide more insight into the variables that cause these actions.